Project 4 - Federated Learning

In [ ]:
!python -m pip install flwr[simulation] 
#flwr_datasets[vision] torch torchvision matplotlib

In [ ]:
!pip install --force-reinstall --no-deps cryptography==41.0.7
#When it gives error

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr as fl
from flwr.common import Metrics
#from flwr_datasets import FederatedDataset

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)
disable_progress_bar()
import torch
import os
import numpy as np

In [ ]:
NUM_CLIENTS = 3 # Trying with smaller amount of clients??? Maybe helps idk
BATCH_SIZE = 32
CLIENT_FOLDER = "/kaggle/input/bdm-dataset/client"

def load_datasets():
    trainloaders = []
    valloaders = []

    for client_id in range(NUM_CLIENTS):
        client_folder = os.path.join(CLIENT_FOLDER, str(client_id))

        train_x = torch.from_numpy(np.load(os.path.join(client_folder, 'trainx.pyp'), allow_pickle=True))
        train_y = torch.from_numpy(np.load(os.path.join(client_folder, 'trainy.pyp'), allow_pickle=True))
        test_x = torch.from_numpy(np.load(os.path.join(client_folder, 'testx.pyp'), allow_pickle=True))
        test_y = torch.from_numpy(np.load(os.path.join(client_folder, 'testy.pyp'), allow_pickle=True))

        train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
        test_dataset = torch.utils.data.TensorDataset(test_x, test_y)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
        valloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

        trainloaders.append(trainloader)
        valloaders.append(valloader)

    return trainloaders, valloaders

trainloaders, valloaders = load_datasets()

In [ ]:
len(trainloaders[0].dataset)

In [ ]:
data, labels = next(iter(trainloaders[0]))
len(labels)

**Defining the model**

In [ ]:
from sklearn.metrics import cohen_kappa_score, f1_score, roc_auc_score

In [ ]:
for batch in trainloaders:
    print(len(batch.dataset[0:][0]), "images")
    #print(batch.dataset[0:][1], "label?")

In [ ]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            images = images.permute(0, 3, 1, 2)
            labels = labels.argmax(dim=1)
            optimizer.zero_grad()
            #print(images.shape, labels.shape)
            outputs = net(images)
            #outputs = outputs.argmax(dim=1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    all_labels = []
    all_predictions = []
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            images = images.permute(0, 3, 1, 2)
            labels = labels.argmax(dim=1)
            #print(images.shape, labels.shape)
            outputs = net(images)
            #outputs = outputs.argmax(dim=1)
            #print(outputs.shape)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            #all_predictions.extend(torch.nn.functional.one_hot(predicted, num_classes=outputs.size(1)).cpu().numpy())
    loss /= len(testloader.dataset)
    accuracy = correct / total
    kappa = cohen_kappa_score(all_labels, all_predictions)#53
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    #roc = roc_auc_score(all_labels, all_predictions, multi_class='ovr')
    #return loss, accuracy, kappa, f1, roc
    print(f"Loss: {loss}, accuracy {accuracy}, kappa {kappa}, f1 {f1}")
    return loss, accuracy, kappa, f1

**Federated learning**

In [ ]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

**Flower Client**

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=3)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        #loss, accuracy, kappa, f1, roc = test(self.net, self.valloader)
        loss, accuracy, kappa, f1 = test(self.net, self.valloader)
        #return float(loss), len(self.valloader), {"accuracy": float(accuracy), "kappa": float(kappa), "f1": float(f1), "roc": float(roc)}
        return float(loss), len(self.valloader), {"accuracy": float(accuracy), "kappa": float(kappa), "f1": float(f1)}

In [ ]:
from torchvision import models

def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    model = models.vgg16()
    input_lastLayer = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(input_lastLayer,10)
    net = model.to(DEVICE)
    #net = models.vgg16().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader).to_client()

In [ ]:

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}
'''def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Initialize sums for all metrics
    loss_sum = accuracy_sum = kappa_sum = f1_sum = 0
    total_examples = 0

    # Compute sums for all metrics
    for num_examples, m in metrics:
        loss_sum += num_examples * m["loss"]
        accuracy_sum += num_examples * m["accuracy"]
        kappa_sum += num_examples * m["kappa"]
        f1_sum += num_examples * m["f1"]
        total_examples += num_examples

    # Compute weighted averages for all metrics
    loss_avg = loss_sum / total_examples
    accuracy_avg = accuracy_sum / total_examples
    kappa_avg = kappa_sum / total_examples
    f1_avg = f1_sum / total_examples

    return {"loss": loss_avg, "accuracy": accuracy_avg, "kappa": kappa_avg, "f1": f1_avg}'''

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=2,  # Never sample less than 10 clients for training
    min_evaluate_clients=2,  # Never sample less than 5 clients for evaluation
    min_available_clients=2,  # Wait until all 10 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Specify the resources each of your clients need. By default, each
# client will be allocated 1x CPU and 0x GPUs
client_resources = {"num_cpus": 1, "num_gpus": 0.0}
if DEVICE.type == "cuda":
    # here we are assigning an entire GPU for each client.
    client_resources = {"num_cpus": 1, "num_gpus": 1.0}
    # Refer to our documentation for more details about Flower Simulations
    # and how to setup these `client_resources`.

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=50),
    strategy=strategy,
    client_resources=client_resources,
)